In [5]:
%cd ../

In [6]:
import os

import pandas as pd
import pandas_datareader.data as web  # For reading stock data from yahoo


class Collector:
    def __init__(self, ticker, start, end, data_path):
        self.ticker = ticker
        self.start = start
        self.end = end
        self.data_path = data_path

    def fetch(self):
        data = web.DataReader(self.ticker, 'yahoo', self.start, self.end)
        data.reset_index(inplace=True)
        data.to_csv(self.data_path)

        return data

    def get_historical(self):
        if os.path.exists(self.data_path):
            data = pd.read_csv(self.data_path, index_col=0)
            if pd.to_datetime(data['Date'][0]) > pd.to_datetime(self.start):
                data = self.fetch()
        else:
            data = self.fetch()

        return data

In [7]:
start = '2004-01-01'
end = '2021-01-01'
ticker = 'BTC-USD'
data_path = os.path.join('data', f'{ticker}.csv')

collector = Collector(ticker, start, end, data_path)
original_data = collector.get_historical()
data = original_data.copy()


In [8]:
pd.read_csv(data_path, index_col=0)

,Date,High,Low,Open,Close,Volume,Adj Close
0,2014-09-16,468.174011,452.421997,465.864014,457.334015,2.105680e+07,457.334015
1,2014-09-17,456.859985,413.104004,456.859985,424.440002,3.448320e+07,424.440002
2,2014-09-18,427.834991,384.532013,424.102997,394.795990,3.791970e+07,394.795990
3,2014-09-19,423.295990,389.882996,394.673004,408.903992,3.686360e+07,408.903992
4,2014-09-20,412.425995,393.181000,408.084991,398.821014,2.658010e+07,398.821014
...,...,...,...,...,...,...,...
2291,2020-12-29,27370.720703,25987.298828,27081.810547,27362.437500,4.526595e+10,27362.437500
2292,2020-12-30,28937.740234,27360.089844,27360.089844,28840.953125,5.128744e+10,28840.953125
2293,2020-12-31,29244.876953,28201.992188,28841.574219,29001.720703,4.675496e+10,29001.720703
2294,2021-01-01,29600.626953,28803.585938,28994.009766,29374.152344,4.073030e+10,29374.152344


In [9]:
data.shape

(2296, 7)

In [10]:
data.head()

,Date,High,Low,Open,Close,Volume,Adj Close
0,2014-09-16,468.174011,452.421997,465.864014,457.334015,21056800.0,457.334015
1,2014-09-17,456.859985,413.104004,456.859985,424.440002,34483200.0,424.440002
2,2014-09-18,427.834991,384.532013,424.102997,394.795990,37919700.0,394.795990
3,2014-09-19,423.295990,389.882996,394.673004,408.903992,36863600.0,408.903992
4,2014-09-20,412.425995,393.181000,408.084991,398.821014,26580100.0,398.821014


In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2296 entries, 0 to 2295
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       2296 non-null   datetime64[ns]
 1   High       2296 non-null   float64       
 2   Low        2296 non-null   float64       
 3   Open       2296 non-null   float64       
 4   Close      2296 non-null   float64       
 5   Volume     2296 non-null   float64       
 6   Adj Close  2296 non-null   float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 125.7 KB


In [12]:
pd.to_datetime(data['Date'][0])

Timestamp('2014-09-16 00:00:00')

In [13]:
import numpy as np
import pandas as pd
from scipy.signal import argrelextrema

In [14]:
def find_support(data, price = 'Close', order = 400):
    support_ix =  argrelextrema(data[price].values, np.less_equal, order = order)
    data['Support'] = data.iloc[support_ix][0][price]
    for i in data[data['Support'].notnull()].index:
        for n in range(-int(order/2), int(order/4)+1, 1):
            step = pd.Timedelta(n, unit = 'h')
            if (i - step) < data.index[-1] and (i - step) > data.index[0]:
                data.loc[i - step, 'Support Line'] = data.loc[i, 'Support']


In [15]:
price = 'Close'
order = 400
support_ix =  argrelextrema(data[price].values, np.less_equal, order = order)
support_ix


(array([ 119, 1550], dtype=int64),)

In [62]:
from sklearn.base import BaseEstimator, TransformerMixin

class BaseTrader(BaseEstimator, TransformerMixin):
    def __init__(self, price = 'Close', valid_days = 50,
                 break_support = 0.1, break_resist = 0.1):
        self.price = price
        self.valid_days = valid_days
        self.dip_to_buy = 1 - break_support
        self.hype_to_sell = 1 + break_resist

    def fit(self, X):
        return X

    def transform(self, X):
        pass
    # ------------- Support and Resistance -------------
    def find_base(self, X, support = True):
        if support:
            base, func = 'Support', np.less_equal
        else:
            base, func = 'Resistance', np.greater_equal
            
        base_ix =  argrelextrema(X[self.price].values, func, order = self.valid_days)
        base_price = X.loc[base_ix, self.price]
        X[base] = base_price
        self.find_baseline(X, base)  # base lines to plot later

        return list(base_price[::-1]), list(base_ix[0][::-1])

    def find_baseline(self, X, base, divide_by = 50):
        line_range = len(X) // divide_by
        X[f"{base} Line"] = X[base].fillna(method='ffill', limit = line_range)
        X[f"{base} Line"].fillna(method='bfill', limit=line_range, inplace=True)

    def find_support(self, X):
        self.find_base(X, support = True)
    
    def find_resistance(self, X):
        self.find_base(X, support = False)


    # ------------- Buy & Sell Signals -------------

    def make_signal(self, X):
        X['Signal'] = 0
        all_bases, base_ix = self.find_base(X)
        curr_bases, bought_bases = [], []
        for i in range(len(X)):
            if len(base_ix) > 0 and i > base_ix[-1]:  # if iterator passes the day of a support
                curr_bases.append(all_bases.pop())  # that base becomes available to be compared
                curr_bases.sort()  # make the last support the biggest one
                base_ix.pop()  # no need to record the date that have been passed

            curr_price = X.loc[i, self.price]
            if len(curr_bases) > 0 and \
                    curr_price < curr_bases[-1] * self.dip_to_buy:
                        X.loc[i, 'Signal'] = 1
                        X.loc[i, 'Bought Price'] = curr_price
                        bought_bases.append(curr_bases.pop())

            if len(bought_bases) > 0 and \
                    curr_price > bought_bases[0] * self.hype_to_sell:
                        X.loc[i, 'Signal'] = -1
                        X.loc[i, 'Sold Price'] = curr_price
                        bought_bases = bought_bases[1:]

In [63]:
data = original_data.copy()
transformer = BaseTrader(break_support=0.2, break_resist=0.5)
transformer.find_base(data)
transformer.make_signal(data)

In [60]:
import plotly.io as pio
pio.renderers.default = "browser"

In [67]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows = 2, cols = 1, shared_xaxes = True, row_heights = [0.8, 0.2])
price = go.Scatter(x = data['Date'], y = data['Close'], name = ticker.upper(),
                   marker_color = '#ffffff')

fig.add_trace(price, row = 1, col = 1)
trace_support = go.Scatter(x = data['Date'], y = data['Support Line'],
                           name = 'Support', mode = 'lines',
                           marker_color = '#ea3943', showlegend=False)
fig.add_trace(trace_support, row = 1, col = 1)
# trace_resistance = go.Scatter(x = data['Date'], y = data['Resistance Line'],
#                            name = 'Resistance', mode = 'lines',
#                            marker_color = '#ea3943')
# fig.add_trace(trace_resistance, row = 1, col = 1)
trace_sellsignals = go.Scatter(x = data['Date'], y = data['Sold Price'],
                           name = 'Sell', mode = 'markers',
                           marker_color = '#ea3943',
                           marker_symbol = 'triangle-down',
                           marker_size = 15)
fig.add_trace(trace_sellsignals)
trace_buysignals = go.Scatter(x = data['Date'], y = data['Bought Price'],
                           name = 'Buy', mode = 'markers',
                           marker_color = '#16c784',
                           marker_symbol = 'triangle-up',
                           marker_size = 15)
fig.add_trace(trace_buysignals, row = 1, col = 1)
volume = go.Bar(x = data['Date'], y = data['Volume'], name = 'Volume',
                opacity = 1, marker_line_width = 0, marker_color = '#808080',
                showlegend=False)
fig.add_trace(volume, row = 2, col = 1)
colors = {
    'background': '#111111',
    'text': '#ffffff'
}

fig.update_layout(
    plot_bgcolor=colors['background'],
    paper_bgcolor=colors['background'],
    font_color=colors['text']
)

fig.update_yaxes(type="log", dtick=0.5, row=1, col=1)
fig.update_xaxes(showgrid=False)
fig


In [56]:
import dash
import dash_core_components as dcc
import dash_html_components as html

external_stylesheets = [
    {
        "href": "https://fonts.googleapis.com/css2?family=Lato:wght@400;700&display=swap",
        "rel": "stylesheet",
    },
]

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

colors = {
    'background': '#111111',
    'text': '#7FDBFF'
}

fig.update_layout(
    plot_bgcolor=colors['background'],
    paper_bgcolor=colors['background'],
    font_color=colors['text']
)


app.layout = html.Div(style={'backgroundColor': colors['background']}, children=[
    html.H1(
        children='Buy the Dip - Sell the Hype',
        style={
            'textAlign': 'center',
            'color': colors['text']
        }
    ),

    dcc.Graph(
        id='example-graph-2',
        figure=fig
    )
])
app.run_server(debug=False)  # Turn off reloader if inside Jupyter

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn in

In [ ]:
trace_support = go.Scatter(x = data.index, y = data['Support Line'],
                           name = 'Support', mode = 'lines',
                           marker_color = '#21CCCB')
trace_resistance = go.Scatter(x = data.index, y = data['Resistance Line'],
                           name = 'Resistance', mode = 'lines',
                           marker_color = '#FF6E58')

trace_buysignals = go.Scatter(x = data.index, y = data['Bought Price'],
                           name = 'Buy', mode = 'markers',
                           marker_color = '#57e51e',
                           marker_symbol = 'triangle-up',
                           marker_size = 15)
trace_sellsignals = go.Scatter(x = data.index, y = data['Sold Price'],
                           name = 'Sell', mode = 'markers',
                           marker_color = '#e53c1e',
                           marker_symbol = 'triangle-down',
                           marker_size = 15)

fig.add_trace(trace_support)
fig.add_trace(trace_resistance)
fig.add_trace(trace_buysignals)
fig.add_trace(trace_sellsignals)
